In [5]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db.models import Sum, Max, Subquery, Q, F, Avg, Count, Min, Max, Sum, F, Q
from django.db.models import Value, IntegerField
from django.db.models.functions import Coalesce, Greatest, Floor
from account.models import User
from booth.models import Transaction, Participation
from player.models import Player

In [48]:
from django.db.models import Sum, Max, Value, IntegerField
from django.db.models.functions import Least, Coalesce, Floor
import pandas as pd

rich_df = pd.DataFrame(Participation.objects \
    .values('player') \
    .annotate(total_score=
              Least(300, Sum('score__health_score') + Max('player__born_health_score')) +
              Least(300, Sum('score__skill_score') + Max('player__born_skill_score')) +
              Least(300, Sum('score__relationship_score') + Max('player__born_relationship_score')) +
              Least(300, Sum('score__growth_score') + Max('player__born_growth_score'))
     ).order_by('-total_score'))
    

In [54]:
parti_money_df = pd.DataFrame(Participation.objects.values('player').annotate(money=Sum('score__money')))
player_df =  pd.DataFrame(Player.objects.values('id', money=F('born_money')).all())
pay = Coalesce(Sum('money', filter=Q(type='pay')), Value(0))
receive = Coalesce(Sum('money', filter=Q(type='receive')), Value(0)) 
deposit = Coalesce(Sum('money', filter=Q(type='deposit')), Value(0))
withdrawal_money = Coalesce(
    Floor(
        Sum(F('money') * (1 + F('interest_rate')), output_field=IntegerField(), filter=Q(type='withdrawal'))
    ), 
    Value(0)
)
withdrawal_deposit = Coalesce(Sum(F('money'), filter=Q(type='withdrawal')), Value(0))
trx_df = pd.DataFrame(Transaction.objects.values('player').annotate(
    money = pay - receive - deposit + withdrawal_money + deposit - withdrawal_deposit,
)
                     )
pd.concat([parti_money_df, player_df, trx_df]).groupby('player', as_index=False)['money'].sum()

,player,money
0,3.0,2220
1,4.0,2450
2,5.0,700
3,10.0,1000
4,12.0,40
5,14.0,2000
6,15.0,100
7,19.0,10000
8,20.0,1501000
9,22.0,6000


In [27]:
parti_money = Participation.objects.values('player').annotate(money=Sum('score__money'))
player =  Player.objects.values(player=F('id'), money=F('born_money')).all()
pay = Coalesce(Sum('money', filter=Q(type='pay')), Value(0))
receive = Coalesce(Sum('money', filter=Q(type='receive')), Value(0)) 
deposit = Coalesce(Sum('money', filter=Q(type='deposit')), Value(0))
withdrawal_money = Coalesce(
    Floor(
        Sum(F('money') * (1 + F('interest_rate')), output_field=IntegerField(), filter=Q(type='withdrawal'))
    ), 
    Value(0)
)
withdrawal_deposit = Coalesce(Sum(F('money'), filter=Q(type='withdrawal')), Value(0))
trx_money =Transaction.objects.values('player') \
                       .annotate(money = pay - receive - deposit + withdrawal_money + deposit - withdrawal_deposit)
# final = parti_money | player | trx_money
# from itertools import chain
parti_money
# player.union(parti_money).union(trx_money).values('player')
# chain(parti_money, trx_money).agg(money=Sum('money'))
# print(final.values('player').agg(money=Sum('money')))
# rich_df = pd.concat([parti_money_df, player_df, trx_df]).groupby('player', as_index=False)['money'].sum()
# rich_df = rich_df.sort_values('money', ascending=False)
# rich_df['player'] = rich_df['player'].astype('int')
# rich_df['money'] = rich_df['money'].astype('int')
# return rich_df[:no_of_rows]

<QuerySet [{'player': 7, 'money': 2000}]>